https://www.kaggle.com/mazenramadan/imdb-most-popular-films-and-series (Kaggle Database)

https://www.rottentomatoes.com/top/bestofrt/ (Rotter tomates top 100 movies)

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

import collections
import matplotlib.pyplot as plt
import seaborn as sns

# Films Rotten Tomatoes

In [ ]:
url = "https://www.rottentomatoes.com/top/bestofrt/"
response = requests.get(url)
response

In [ ]:
soup = BeautifulSoup(response.content, features = "lxml")
#print(soup.prettify())

In [ ]:
movies = soup.find_all("table", {"class":"table"})
movies

data = []

for movie in movies:
    for row in movie.findAll('tr')[1:]:
        
        movie_names = row.findAll("a", {"class":"unstyled articleLink"})
        movie_name = movie_names[0].text.strip().capitalize()
        movie_name = movie_name[:len(movie_name) - 7]
        movie_name = re.sub('[^A-Za-z0-9 ]+', '', movie_name)
        
        movie_ratings = row.findAll("span", {"class":"tMeterScore"})
        movie_rating = movie_ratings[0].text
        movie_rating = re.sub('[^0-9 ]+', '', movie_rating)
        
        data.append({"Film name": movie_name, "Film Rating Rotten Tomatoes" : movie_rating})

TopTomatoes = pd.DataFrame(data)
TopTomatoes["Film Rating Rotten Tomatoes"] = TopTomatoes["Film Rating Rotten Tomatoes"].apply(pd.to_numeric)
TopTomatoes

In [ ]:
# Create the top 10 movies from Rotter Tomatoes

TopTomatoes = TopTomatoes.sort_values(by=["Film Rating Rotten Tomatoes"], ascending= False)
Top10Tomates = TopTomatoes.head(10)
Top10Tomates

# Working on the IMDB data

In [ ]:
#IMDM datasource
imdb = pd.read_csv("C:\\Users\\beatr\\Documents\\Beatriz\\Ironhack\\Projects\\Final Project - Module 1\\Project-Week-3-Data-Thieves\\your-project\\imdb.csv")
imdb

In [ ]:
#IMDB data cleaning
imdb = imdb.drop_duplicates(subset=['Name'])
imdb = imdb[imdb["Type"] == "Film"]
imdb = imdb[imdb["Rate"] != "No Rate"]
imdb

In [ ]:
#imdb.info() - Verify that both columns Values and Rates were objects

imdb.Votes = imdb.Votes.str.replace(",", "") #to replace the comma in the Votes column

imdb[["Votes", "Rate"]] = imdb[["Votes", "Rate"]].apply(pd.to_numeric) #change the type of the both columns
imdb

In [ ]:
imdb = imdb[imdb["Votes"] >= 25000]
imdb

In [ ]:
imdb["Name"] = imdb["Name"].str.replace(r'[^A-Za-z0-9 ]+', "")
imdb

In [ ]:
#Create a new data set with only the necessary columns

new_imdb = imdb[["Name", "Rate", "Genre"]]
new_imdb

In [ ]:
#Replace those movies write wrong names
new_imdb["Name"] = new_imdb["Name"].replace(to_replace ="Gisaengchung", value ="Parasite Gisaengchung")
new_imdb["Name"] = new_imdb["Name"].replace(to_replace ="Shichinin no samurai", value ="Seven Samurai Shichinin no Samurai")
new_imdb["Name"] = new_imdb["Name"].replace(to_replace ="Sunset Blvd", value ="Sunset Boulevard")
new_imdb

#new_imdb.to_excel("new_imdb.xlsx")

In [ ]:
new_imdb = new_imdb.sort_values(by=["Rate",'Name'], ascending=False)
new_imdb['Name'] = new_imdb['Name'].str.capitalize()

In [ ]:
#Creating our own IMDB top 100 

TopImdb = new_imdb.head(100)

TopImdb = TopImdb.rename(columns={"Name": "Film name", "Rate": "Film Rating IMDB", "Genre": "Genre"})
TopImdb

#TopImdb.to_excel("TopImdb.xlsx")  

In [ ]:
#Inner Join:
Bothrakings = pd.merge(TopImdb, TopTomatoes, left_on='Film name', right_on='Film name')
Bothrakings.head(100)

#The critics and the viewers have different movies tastes (we found only 15 movies matching both)

In [ ]:
#Creating our own IMDB top 10

Top10Imdb = new_imdb.head(10)
Top10Imdb

# Top Genres of IMDB

In [ ]:
#TopImdb["Genre"] = TopImdb["Genre"].apply(lambda x: list(x.split(",")))
TopImdb["Genre"]

list_genres_imdb =[] 

for item in TopImdb["Genre"]:
    list_genres_imdb.append(item)

list_new = []

for element in list_genres_imdb:
    #print(type(element))
    element = element.split(",")
    #print(element)
    list_new.append(element)
    
list_newnew = [word.upper().strip() for element in list_new for word in element]
    
print(list_newnew)

In [ ]:
# initialize a null list
unique_list = []
     
# traverse for all elements
for x in list_newnew:
    # check if exists in unique_list or not
    if x not in unique_list:
        unique_list.append(x)
    
print(unique_list)

In [ ]:
occurrences = collections.Counter(list_newnew)
occurrences

In [ ]:
df_genres_imdb = pd.DataFrame.from_dict(occurrences, orient='index').reset_index()
df_genres_imdb = df_genres_imdb.rename(columns={'index':'Genres IMDB', 0:'count'})
df_genres_imdb = df_genres_imdb.sort_values(by=["count"], ascending=False)
df_genres_imdb

In [ ]:
df_genres_imdb['Genres IMDB'] = df_genres_imdb['Genres IMDB'].str.capitalize()
df_genres_imdb

In [ ]:
top_genres_imdb = df_genres_imdb.head(3)
top_genres_imdb

In [ ]:
top_genres_imdb = top_genres_imdb.set_index('Genres IMDB')
top_genres_imdb

In [ ]:
top_genres_imdb.plot(kind='bar')
plt.title('Top 3 genres on IMDB', color = 'red', size = '20')
plt.xticks(rotation=0)
plt.ylabel('Popularity')
plt.show()

# Top Genres Rotten Tomatoes

In [1]:
# Extracting link for every movie from rotten tomatoes top 100
movie_url_list = []
for movie in movies:
    for row in movie.findAll('tr')[1:]:
        
        movie_link = row.a['href']
        movie_url = 'https://www.rottentomatoes.com' + row.a['href']
        movie_url_list.append(movie_url)
movie_url_list

NameError: name 'movies' is not defined

In [ ]:
# Extracting genres of movies from rotten tomatoes top 100
rotten_genres = []
for link in movie_url_list:
    try:
        response = requests.get(link)
        soup = BeautifulSoup(response.content, features = 'lxml')
        # get the genre
        genre_link = soup.find_all('div', attrs = {'class': 'meta-value genre'})
        genre = genre_link[0].text.strip()
        genre = re.sub('[^A-Za-z0-9&,/ ]+', '', genre)
        genre = re.sub(' ', '', genre)
    
        # putting found genres into a list
        rotten_genres.append(genre)
    except:
        print(f'issue with this link - {link}')
print(rotten_genres)

In [ ]:
# We received list with multiple genres for each movie, trying to clean it better
rot_gen = []
for item in rotten_genres:
    item = item.split(',')
    rot_gen.append(item)
print(rot_gen)

In [ ]:
r_g = []
for string in rot_gen:
    for word in string:
        word = word.split(',')
        #word = word.split('&')
        r_g.append(word)
print(r_g)

In [ ]:
# Finally a list with genres from rotten tomatoes
rg = [i for element in r_g for i in element]
print(rg)

In [ ]:
# Creating a dictionary with frequency of each genre in rotten tomatoes top 100
genre_freq={}
for word in rg:
    if word in genre_freq:
        genre_freq[word]+=1
    else:
        genre_freq[word]=1
print(genre_freq)

In [ ]:
# Here we have a dataframe to visualize genres

data_items = genre_freq.items()
data_list = list(data_items)

rottentotomatoes_genres = pd.DataFrame(data_list, columns = ['Genre', 'Popularity'])
rottentotomatoes_genres

In [ ]:
# Our goal to see top 3 genres what critics choose

rottentotomatoes_genres = rottentotomatoes_genres.sort_values(by=['Popularity'], ascending=False)
rottentotomatoes_genres = rottentotomatoes_genres.head(3)
rottentotomatoes_genres

In [ ]:
# Moving column Genre as an index for better vizualization on chart

rottentotomatoes_genres = rottentotomatoes_genres.set_index('Genre')

In [ ]:
# Chart of Top 3 genres on Rotten Tomatoes

rottentotomatoes_genres.plot(kind='bar')
plt.title('Top 3 genres on Rotten Tomatoes', color = 'red', size = '20')
plt.xticks(rotation=0)
plt.ylabel('Popularity')
plt.show()